In [8]:
import sys
sys.path.append('../..')

import attack
import data_loader as dl

In [40]:
import geopandas as gp
import pandas as pd
from tqdm import tqdm
import numpy as np
import libpysal
import itertools
from joblib import Parallel, delayed
import random
import matplotlib.pyplot as plt
from shapely import Point

In [34]:
# Load data
gdf = gp.read_file('full_trip_HL_clustering.geojson')

In [35]:
# format clusering_HL columns as object
gdf['clustering_HL'] = gdf['clustering_HL'].astype('object')

In [49]:
gdf['START_POINT'] = gdf.geometry.apply(lambda x: Point(x.coords[0]))
gdf['END_POINT'] = gdf.geometry.apply(lambda x: Point(x.coords[-1]))

In [50]:
# plot trajectories of a random user and the corresponding clusters
user = random.choice(gdf['PERSON_ID'].unique())


# select random user
user_gdf = gdf[gdf['PERSON_ID'] == user]

# print nr of trajectories of user
print('Nr of trajectories of user: ', len(user_gdf))

# print nr of clusters of user
print('Nr of clusters of user: ', len(user_gdf['clustering_HL'].unique()))

# plot trajectories of user
user_gdf.explore(column='clustering_HL')


Nr of trajectories of user:  31
Nr of clusters of user:  10


c:\Users\Bened\Documents\Git\Master-Thesis\env\lib\site-packages\geopandas\explore.py:362: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  cat = pd.Categorical(gdf[column][~nan_idx], categories=categories)


In [56]:
# plot start and end points of user
gp.GeoDataFrame(user_gdf[['START_POINT', 'END_POINT', 'clustering_HL']], geometry='START_POINT').explore(column='clustering_HL')

c:\Users\Bened\Documents\Git\Master-Thesis\env\lib\site-packages\geopandas\explore.py:362: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  cat = pd.Categorical(gdf[column][~nan_idx], categories=categories)


In [58]:
points = pd.read_pickle('../../data/freemove/freemove_point_geographical_context.pickle')
points['lat'] = points.geometry.apply(lambda x: x.y)
points['lng'] = points.geometry.apply(lambda x: x.x)

In [61]:
from skmob.tessellation import tilers
import skmob
tessellation = tilers.tiler.get("squared", base_shape='Berlin, Germany', meters=500)

tdf = skmob.TrajDataFrame(points)
# mapped = tdf.mapping(tessellation, remove_na=True)

ValueError: linearrings: Input operand 0 does not have enough dimensions (has 1, gufunc core with signature (i, d)->() requires 2)